# Week 09: Word Sense Disambiguation

This week, we introduced a hot topic in Natural Language Proccessing: *Word Sense Disambiguation (WSD)* .  
Many words in natural languages have ambiguous meanings. For example, the word *[party](https://dictionary.cambridge.org/dictionary/english/party)* can refer to 1) a social gathering (派對), 2) a political organization (政黨), or 3) an entity in law (當事人；⋯⋯方).  
As a human, we can distinguish different meanings easily, but can a machine do the same? This is what WSD aims for.  

## Introduction

### tl; dr
You have to 
1. preprocess the data
2. (stage 1) generate a small training dataset from the given collocation seed,
3. (stage 1) train a weak model on that small dataset,
4. (stage 2) use the weak model to generate more labeled data, and
5. (stage 2) train your final model
6. Evaluate your model on testing data (requirement: accuracy > 0.7)

### Concept

In [Lesk's assumption](https://en.wikipedia.org/wiki/Lesk_algorithm), each word has only one sense when it appears in the same collocation.  
For example, if *party* shows up with the word *court* (法庭), most likely the sense of this *party* is the 3rd one: an entity in law (當事人；⋯⋯方).  
However, we are not implementing Lesk's algorithm this week. Instead, we will combine his assumption with [Yarowsky's](https://en.wikipedia.org/wiki/Yarowsky_algorithm) *bootstrap technique* .  

You are given some pre-defined collocations, or called *seeds*, of the word *party*, along with which sense each collocation belongs to.  
With the given seeds, you can generate a small set of labeled data by rule. Then with this small set, we can train a small model with limited accuracy.  
The current classifier might not perform well on the whole dataset, sure, but it's already enough to generate more reliable labeled data. With the newly labeled training data, we can now train another sense-classification model with more robustness, which aims for the real WSD task.  
This process, about training on smaller dataset, generating more data, and then improving the model itself, is called *[bootstrapping](https://www.mastersindatascience.org/learning/introduction-to-machine-learning-algorithms/bootstrapping/)* .  


In [2]:
!pip install googledrivedownloader
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1ekUZ1zGSs6UjM_jfmAOuzNCgcOyWdsyR',
                                    dest_path='./GoogleNews-vectors-negative300.bin.gz')

In [3]:
import gzip
import shutil
with gzip.open('GoogleNews-vectors-negative300.bin.gz', 'rb') as f_in:
    with open('GoogleNews-vectors-negative300.bin', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

<a name="I.-Data-preparation"></a>
## I. Data preparation

First thing first. To make natural language understandable for machines, we have to transform sentences into embeddings.  
So here are four things to do:

1. load data
2. preprocess the sentences
3. transform sentences into embeddings
4. pad the sentences to the same length

To make the task simple and easy to understand, we will only work on a single word *party* .  
Three senses of *party* is defined as below with their corresponding `sense id`s. 

In [4]:
SENSE = {
    1: 'a social event at which a group of people meet to talk, eat, drink, dance, etc.', # 派對
    2: 'an organization of people with particular political beliefs', # 政黨
    3: 'a single entity which can be identified as one for the purposes of the law' # （法庭）當事人；⋯⋯方
}

### 1. Load data

The data is a set of sentences containing the word *party*, all extracted from wikipedia. The uniqueness of each sentence is guaranteed. 

In [5]:
import os

In [6]:
with open(os.path.join('.', 'party.train.txt'), 'r') as f:
    data = f.read().strip().split('\n')

# this dict maps sentence_id to the sentence itself
pure_data = { sent_id: text for sent_id, text in [line.split('\t', 1) 
                                                 for line in data] }

Let's see what the data looks like.

In [7]:
for sent_id, sentence in pure_data.items():
    if int(sent_id) > 1003: break
        
    print(f'{sent_id}: {sentence}')

1001: A naked party, also known as nude party, is a party where the participants are required to be nude.
1002: The town center bears the hallmarks of a typical migration-accepting Turkish rural town, with traditional structures coexisting with a collection of concrete apartment blocks providing public housing, as well as amenities such as basic shopping and fast-food restaurants, and essential infrastructure but little in the way of culture except for cinemas and large rooms hired out for wedding parties.
1003: Elections Alberta oversees the creation of political parties and riding associations, compiles election statistics on ridings, and collects financial statements from party candidates and riding associations.


In [8]:
# a look up table from sentence to id
id_mapper = {v: k for k, v in pure_data.items()}
# a table for id to embedding; we will deal with this later
processed_data = {}

We define 2 samples here to validate the preprocess during our coding.

In [9]:
samples = [
    'Adnan Al-Hakim (died May 26, 1990) was the leader of the Najjadeh Party, an Arab nationalist party in Lebanon, for more than 30 years.',
    'A block party or street party is a party in which many members of a single community congregate, either to observe an event of some importance or simply for mutual enjoyment.'
]

### 2. Preprocess the sentences 

<font color="red">[TODO]</font> Define your preprocessing function to transform a sentence into tokens here.  

\-

<small>
*hint: If you can't get a high accuracy in the final result, you may want to come back and modify your preprocessing here.<br/>
*hint: Think about what words are useful and what are useless when distinguishing a sense.
</small>

In [10]:
import re
def preprocess(text):
    text = re.findall("[\w']+", text)
    return text

In [11]:
sent_tokens = [preprocess(sent) for sent in samples]
sent_tokens[0][:5]

['Adnan', 'Al', 'Hakim', 'died', 'May']

### 3. Transform sentences into embeddings

For the simplicity, we are still using word2vec here, so you can copy-paste your code from previous week.  
This is not required; you don't have to use word2vec if you want to train a embedding model along with the classifier.  

<small>\*Download w2v: [Google Code Archive](https://code.google.com/archive/p/word2vec/#Pretrained-word-and-phrase-vectors)</small>

In [12]:
import numpy as np
from gensim.models import KeyedVectors

In [13]:
w2v = KeyedVectors.load_word2vec_format(
        os.path.join('.', 'GoogleNews-vectors-negative300.bin'), 
        binary = True
      )

In [14]:
import numpy

In [15]:
def to_embedding(tokens):
    embedding_dim=300
    num_tokens=[]
    for word in tokens:
        if word in w2v:
            num_tokens.append(word)
    embedding_matrix = numpy.zeros((len(num_tokens), embedding_dim))
    for i in num_tokens:
        embedding_matrix[num_tokens.index(i)] = w2v[i]
    return embedding_matrix

In [16]:
embeddings = [to_embedding(tokens) for tokens in sent_tokens]
embeddings[0]

array([[-0.140625  ,  0.20703125, -0.12988281, ...,  0.03076172,
         0.07080078,  0.484375  ],
       [-0.0390625 ,  0.24804688,  0.00540161, ...,  0.2265625 ,
         0.02404785, -0.01477051],
       [ 0.05541992,  0.31640625,  0.27929688, ..., -0.06689453,
         0.34179688,  0.27929688],
       ...,
       [ 0.06396484, -0.25585938, -0.08447266, ...,  0.02746582,
         0.06494141,  0.06201172],
       [-0.07666016, -0.10400391, -0.00175476, ..., -0.01965332,
        -0.03442383,  0.0007515 ],
       [-0.12695312,  0.20898438, -0.10644531, ...,  0.13476562,
         0.01879883, -0.1484375 ]])

### 4. Pad the sentences to the same length

The input size of model is fixed. However, the sentence lengths are various.  
An intuitive solution is to stuff some dummy values into arrays util they share the same size, and this is called *padding*.  

<small>*<a href="https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences">tf.keras.preprocessing.sequence.pad_sequences</a></small>

In [17]:
# if you prefer numpy
#import numpy as np
# or if you prefer tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [18]:
embeddings

[array([[-0.140625  ,  0.20703125, -0.12988281, ...,  0.03076172,
          0.07080078,  0.484375  ],
        [-0.0390625 ,  0.24804688,  0.00540161, ...,  0.2265625 ,
          0.02404785, -0.01477051],
        [ 0.05541992,  0.31640625,  0.27929688, ..., -0.06689453,
          0.34179688,  0.27929688],
        ...,
        [ 0.06396484, -0.25585938, -0.08447266, ...,  0.02746582,
          0.06494141,  0.06201172],
        [-0.07666016, -0.10400391, -0.00175476, ..., -0.01965332,
         -0.03442383,  0.0007515 ],
        [-0.12695312,  0.20898438, -0.10644531, ...,  0.13476562,
          0.01879883, -0.1484375 ]]),
 array([[-0.10595703,  0.21386719,  0.11865234, ...,  0.10693359,
          0.02368164, -0.03540039],
        [ 0.11328125,  0.06835938,  0.25      , ...,  0.03564453,
          0.07226562,  0.01519775],
        [-0.09130859, -0.08691406, -0.01208496, ...,  0.02832031,
         -0.23242188,  0.11962891],
        ...,
        [-0.01177979, -0.04736328,  0.04467773, ...,  

In [25]:
def add_padding(embeddings):
    # [ TODO ]
    # Pad all embeddings to padding_width, or detect it automatically when it's not given
    # ps. tensorflow's `pad_sequences` can detect that for you
    embeddings = [sent.tolist() for sent in embeddings]
    embeddings = pad_sequences(embeddings, padding='pre', value=[0]*300, dtype='float32')
    return embeddings

In [20]:
emb_padded = add_padding(embeddings)
emb_padded[0]

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.06396484, -0.25585938, -0.08447266, ...,  0.02746582,
         0.06494141,  0.06201172],
       [-0.07666016, -0.10400391, -0.00175476, ..., -0.01965332,
        -0.03442383,  0.0007515 ],
       [-0.12695312,  0.20898438, -0.10644531, ...,  0.13476562,
         0.01879883, -0.1484375 ]], dtype=float32)

In [21]:
print(len(embeddings[0]), len(embeddings[1]))
print(emb_padded[0].shape, emb_padded[1].shape)

20 26
(26, 300) (26, 300)


You should see the embedding of shorter sentence is padded by empty arrays, and they are at the same length now.

In [22]:
# record the width for the future use.
PADDING_WIDTH = emb_padded[0].shape[0]
print(PADDING_WIDTH)

26


### 5. all-in-one

Define a function to setup the pipeline, and transform all sentences into embeddings!  

<small>\*Your embedding shape might not be the same with ours due to our different preprocessing procedure. </small>

In [29]:
def process_text(sentences):
    result = [ preprocess(sentence) for sentence in sentences ]
    result = [ to_embedding(sentence) for sentence in result ]
    result = add_padding(result)
    return result

In [30]:
X = process_text(pure_data.values())

In [32]:
X[0]

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.12890625, -0.18261719,  0.10351562, ..., -0.07714844,
        -0.11572266, -0.02832031],
       [-0.22851562, -0.08837891,  0.12792969, ..., -0.21289062,
         0.18847656, -0.14550781],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [33]:
X.shape # should be (637, *, 300), * depends on your preprocessing

(637, 84, 300)

Let's use a dictionary to store all embeddings with their sentence_id.

In [34]:
processed_data = { 
    sent_id: embedding for sent_id, embedding in zip(pure_data, X) 
}

In [35]:
print(pure_data['1001'])
processed_data['1001']

A naked party, also known as nude party, is a party where the participants are required to be nude.


array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.12890625, -0.18261719,  0.10351562, ..., -0.07714844,
        -0.11572266, -0.02832031],
       [-0.22851562, -0.08837891,  0.12792969, ..., -0.21289062,
         0.18847656, -0.14550781],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

## II. First stage

After preprocessing the training data, now we are going to train our first-stage model!  

According to the method described at the beginning, we can train a simple model on a smaller dataset, and this dataset can be generated by rule from seeds.  

### Steps

1. Prepare the training data
2. Encode labels
3. Split training and testing dataset
4. Build classifier
5. Train

### 1. Prepare the training data

Given the seed collocationss, you can add a sentence into the training data with label if that sentence contains that collocation.  
For example, we can say <i>"A party is a **social** gathering."</i> should be the first sense, because it contains the keyword *social*. Hence, your training data will have this sentence with its label `1`.  

Don't worry about the false-positive cases for now.  
If the seed is generally good enough, the model will learn to ignore those wrong data by itself. (though yeah, you can get better results if you deal with it beforehand)

In [36]:
SEEDS = {
    1: ['social', 'events'],
    2: ['system', 'coalition'],
    3: ['court', 'law']
}

<font color="red">[TODO]</font> Get the initial training data from the given seeds.  

In [176]:
# [TODO]
indice, first_X, first_Y = [], [], [] # sentence id of selected samples, selected sentences, detected labels
for sent_id, sentence in pure_data.items():
    for key, value in SEEDS.items():
        if (value[0] in sentence) or (value[1] in sentence):
            indice.append(sent_id)
            first_X.append(sentence)
            first_Y.append(key)
first_X = process_text(first_X)

    

Examine training data.  
The labels might not be 100% correct, but it should look reasonable.  

In [177]:
for i in range(5):
    print(pure_data[indice[i]])
    print(f' -> {first_Y[i]}: {SENSE[first_Y[i]]}')
    print()

From these social conventions derive in turn also the variants worn on related occasions of varying solemnity, such as formal political, diplomatic, and academic events, in addition to certain parties including award ceremonies, balls, fraternal orders, high school proms, etc.
 -> 1: a social event at which a group of people meet to talk, eat, drink, dance, etc.

The Free-minded People's Party () or Radical People's Party was a social liberal party in the German Empire, founded as a result of the split of the German Free-minded Party in 1893.
 -> 1: a social event at which a group of people meet to talk, eat, drink, dance, etc.

Typically, a party has the right to object in court to a line of questioning or at the introduction of a particular piece of evidence.
 -> 3: a single entity which can be identified as one for the purposes of the law

Dizzy bat is commonly played at parties, colleges and universities, bars, and other drinking festivities such as a tailgate party at sporting eve

Transform X and Y into numpy array for future use.

In [178]:
first_X = np.array(first_X)
first_Y = np.array(first_Y)
first_X.shape

(201, 84, 300)

### 2. Encode labels

The labels now are all categorical, which are `1`, `2`, and `3` . However, it's hard to teach a machine this kind of answers.  
Most of the time, machine learning generates a *numeric probability*, like `0.329`, rather than a categorical result.  
That's why we want to encode the label into a floating point between 0 ~ 1, so that the machine can generate the probability of each answer.  

Here we suggest you use the one-hot encoding, which is suitable for categorical classification.  
So the label `2` will look like
```
 Sense 1, Sense 2, Sense 3
[      0,       1,       0]
```

*<small><a href="https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/">Why One-Hot Encode Data in Machine Learning?</a></small>

In [179]:
# if you prefer tensorflow
from tensorflow import one_hot
# or if you don't like tensorflow
from sklearn.preprocessing import OneHotEncoder

<font color="red">[TODO]</font> one-hot encode `first_Y`

<small>
*<a href="https://www.tensorflow.org/api_docs/python/tf/one_hot">tf.one_hot</a><br/>
*<a href="https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html">sklearn.preprocessing.OneHotEncoder</a>
</small>

In [180]:
a = first_Y.tolist()
a = [(i-1) for i in a]

In [181]:
depth = 3
first_Y = one_hot(a, depth, axis=-1)

In [182]:
first_Y

<tf.Tensor: shape=(201, 3), dtype=float32, numpy=
array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
     

### 3. Prepare training and validation set

Split the dataset into training set and validation set.  
The reason for splitting is because, you may not want the model to see what you'll use to test it when it is still learning.

Machine is very smart; sometimes it just *memorizes* the answers, rather than *learns* them. Even that the model has yielded a perfect accuracy in the test, it still might fail miserably when facing the cruel, real world. *(heh)*  
That's why we need a validation set. We reserve a partition of data that will never be learnt by the model, and use it to validate whether the model really learns someting.

<small>*<a href="https://tarangshah.com/blog/2017-12-03/train-validation-and-test-sets/">Train, Validation and Test Sets</a></small>

In [45]:
# if you prefer sklearn
from sklearn.model_selection import train_test_split
# or if you don't like sklearn. **Remember to shuffle your data before splitting.**
import numpy as np

In [183]:
print(type(first_Y))
first_Y=first_Y.numpy()

<class 'tensorflow.python.framework.ops.EagerTensor'>


In [184]:
X_train, X_val, Y_train, Y_val = train_test_split(
    first_X, first_Y,
    test_size = 0.2,   # [TODO] How much data you want to used as validation set
    shuffle = True
)

In [185]:
print(X_train.shape, X_val.shape, Y_train.shape, Y_val.shape)

(160, 84, 300) (41, 84, 300) (160, 3) (41, 3)


### 4. Build your multi-labeling classifier 

Now the data is all prepared.  
Let's build a model to learn from it!  

Note that, different from last week, your output dimension should be the size of all categories, rather than `2` .  

\-

<small>
*Although tensorflow is used below, you can always change it to any other framework you are familiar with. <br/>
*<a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers">tf.keras.layers</a>
</small>

In [49]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense#, and all the other layers you may use

In [186]:
_, PADDING_WIDTH, EMBEDDING_DIM = X_train.shape
OUTPUT_CATEGORY = len(SENSE)

print(PADDING_WIDTH, EMBEDDING_DIM, OUTPUT_CATEGORY)

84 300 3


<font color="red">[TODO]</font> Build a classifier

In [187]:
model = Sequential()

model.add(LSTM(32))
model.add(Dense(3, activation='softmax'))
input_shape = (None, X_train.shape[1], X_train.shape[2])
model.build(input_shape)

In [188]:
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 32)                42624     
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 42,723
Trainable params: 42,723
Non-trainable params: 0
_________________________________________________________________
None


Time to choose the optimizer and loss function.  

Loss function is an equation evaluating how wrong your model has answered (the lower the better), while optimizer tells the model how to improve itself.  
But seriously, we are not asking you to fine-tune these parameters. That is for Machine Learning class, not for NLP class, so if you are not able to pass the baseline, go check your processing procedure first. Something might go wrong there.  

\-

<small>
*<a href="https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile">tf.keras.model#compile</a> <br/>
*<a href="https://www.tensorflow.org/api_docs/python/tf/keras/optimizers">tf.keras.optimizers</a> <br/>
*<a href="https://www.tensorflow.org/api_docs/python/tf/keras/losses">tf.keras.losses</a>
</small>

<font color="red">[TODO]</font> Compile your model

In [189]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### 5. Train 

Time to train your model!  

You should always prevent the model from overfitting, so take validation accuracy into consideration and choose your epoch number wisely.  

<small>*<a href="https://www.ibm.com/cloud/learn/overfitting">What is Overfitting?</a></small>

<font color="red">[TODO]</font> Train and tune your model

In [190]:
history = model.fit(
    X_train, Y_train, 
    validation_data=(X_val, Y_val),
    epochs = 15          # [TODO] how many iterations you want to run
    #initial_epoch = 3    # set this if you're continuing previous training
)

Epoch 1/15
5/5 [==============================] - 2s 137ms/step - loss: 1.0322 - accuracy: 0.4812 - val_loss: 0.9882 - val_accuracy: 0.5122
Epoch 2/15
5/5 [==============================] - 0s 43ms/step - loss: 0.9313 - accuracy: 0.5125 - val_loss: 0.9392 - val_accuracy: 0.4878
Epoch 3/15
5/5 [==============================] - 0s 32ms/step - loss: 0.8446 - accuracy: 0.5250 - val_loss: 0.9024 - val_accuracy: 0.4878
Epoch 4/15
5/5 [==============================] - 0s 31ms/step - loss: 0.7762 - accuracy: 0.6125 - val_loss: 0.8534 - val_accuracy: 0.5854
Epoch 5/15
5/5 [==============================] - 0s 35ms/step - loss: 0.6890 - accuracy: 0.7312 - val_loss: 0.7896 - val_accuracy: 0.6585
Epoch 6/15
5/5 [==============================] - 0s 34ms/step - loss: 0.5988 - accuracy: 0.7688 - val_loss: 0.7118 - val_accuracy: 0.6585
Epoch 7/15
5/5 [==============================] - 0s 33ms/step - loss: 0.5006 - accuracy: 0.7812 - val_loss: 0.6047 - val_accuracy: 0.6829
Epoch 8/15
5/5 [==========

### 6. Examine your model

Let's see how good your model does.  

In [191]:
testcases = [
    # 1
    'A block party or street party is a party in which many members of a single community congregate, either to observe an event of some importance or simply for mutual enjoyment.',
    'A party is a social gathering.',
    # 2
    'Ukraine has a multi-party system, with numerous parties in which often not a single party has a chance of gaining power alone, and parties must work with each other to form coalition governments.',
    'Serbia has a multi-party system, with numerous parties in which no one party often has a chance of gaining power alone, and parties must work with each other to form coalition governments.',
    # 3
    'In a civil lawsuit, a nominal party is one named as a party on the record of an action, but having no interest in the action.',
]

In [192]:
# you must specify the padding width here, since the input size of model should always be the same
test_X = process_text(testcases)

In [193]:
predictions = model.predict(test_X)

In [194]:
predictions[0]

array([0.5890655 , 0.10217339, 0.30876106], dtype=float32)

#### What does the result mean?

As you can see, a list of floats are generated, and since we used one-hot encoding when preparing the training data, each number presents the result of corresponding categories.  
```
 Sense 1, Sense 2, Sense 3
[   0.89,    0.12,    0.21]
```
You can consider these values as the probability of each column, or said category. Hence, the true predicted label should be the one with the highest probability, which is Sense 1 for this sample.  

Now let's get all the predicted labels from these probabilities.  

In [195]:
for idx, result in enumerate(predictions):
    predict_id = result.argmax() # select the index of the maximum value
    sense_id = predict_id + 1    # sense_id starts from 1
    print(testcases[idx])
    print(f'-> Sense {sense_id} (prob={result[predict_id]:.2f}): {SENSE[sense_id]}')
    print()

A block party or street party is a party in which many members of a single community congregate, either to observe an event of some importance or simply for mutual enjoyment.
-> Sense 1 (prob=0.59): a social event at which a group of people meet to talk, eat, drink, dance, etc.

A party is a social gathering.
-> Sense 1 (prob=0.52): a social event at which a group of people meet to talk, eat, drink, dance, etc.

Ukraine has a multi-party system, with numerous parties in which often not a single party has a chance of gaining power alone, and parties must work with each other to form coalition governments.
-> Sense 2 (prob=1.00): an organization of people with particular political beliefs

Serbia has a multi-party system, with numerous parties in which no one party often has a chance of gaining power alone, and parties must work with each other to form coalition governments.
-> Sense 2 (prob=1.00): an organization of people with particular political beliefs

In a civil lawsuit, a nominal

Again, the label might not be 100% correct, but it should look reasonable somehow.  

## III. Second stage

The previous model might not be enough for real-world use; another model with better ability is needed.  

<small>*Most contents of this section are the same as previous one, so you can make use of your code above.</small>

### 1. Prepare the training data 

The model from the previous section is weak, yet it still has learned some valuable knowledge.  
Let's ask that model to label more training data for us!

In [196]:
# Get the probability on the whold dataset
predictions = model.predict(np.array(list(processed_data.values())))


<font color="red">[TODO]</font> Get the labels of all data, and reserve only those labels with high probabilities.

In [197]:
THRESHOLD = 0.5  # you may want to change this :)
indice, second_X, second_Y = [], [], [] # sentence id of selected samples, selected sentences, detected labels

for sent_id, result in zip(processed_data, predictions):
    if result[0] >= THRESHOLD :
        indice.append(sent_id)
        second_X.append(processed_data[sent_id])
        second_Y.append(1)
    elif result[1] >= THRESHOLD :
        indice.append(sent_id)
        second_X.append(processed_data[sent_id])
        second_Y.append(2)
    elif result[2] >= THRESHOLD :
        indice.append(sent_id)
        second_X.append(processed_data[sent_id])
        second_Y.append(3)

Observe the selected data size and the quality of labels.  
You might want to go back and modify your preprocessing, first model, or the threshold until you get a better training data.

In [198]:
for i in range(5):
    print(pure_data[indice[i]])
    print(f' -> {second_Y[i]}: {SENSE[second_Y[i]]}')
    print()

The town center bears the hallmarks of a typical migration-accepting Turkish rural town, with traditional structures coexisting with a collection of concrete apartment blocks providing public housing, as well as amenities such as basic shopping and fast-food restaurants, and essential infrastructure but little in the way of culture except for cinemas and large rooms hired out for wedding parties.
 -> 1: a social event at which a group of people meet to talk, eat, drink, dance, etc.

Elections Alberta oversees the creation of political parties and riding associations, compiles election statistics on ridings, and collects financial statements from party candidates and riding associations.
 -> 2: an organization of people with particular political beliefs

From these social conventions derive in turn also the variants worn on related occasions of varying solemnity, such as formal political, diplomatic, and academic events, in addition to certain parties including award ceremonies, balls, 

In [199]:
second_X = np.array(second_X)
second_Y = np.array(second_Y)
second_X.shape

(570, 84, 300)

### 2. Encode labels 

<font color="red">[TODO]</font> one-hot encode secone_Y

In [200]:
second_Y_list = second_Y.tolist()
second_Y_list = [(i-1) for i in second_Y_list]
second_Y = one_hot(second_Y_list, depth, axis=-1)

In [201]:
second_Y[:3]

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], dtype=float32)>

### 3. Prepare training and validating dataset

In [202]:
second_Y=second_Y.numpy()
X_train, X_val, Y_train, Y_val = train_test_split(
    second_X, second_Y,
    test_size = 0.2,    # [TODO] How much data you want to used as validation set
    shuffle = True
)

In [203]:
X_train.shape

(456, 84, 300)

### 4. Build model

In [204]:
# the number comes from previous setting
print(PADDING_WIDTH, EMBEDDING_DIM, OUTPUT_CATEGORY)

84 300 3


<font color="red">[TODO]</font> Build your second model

<small>*This model can be different from the previous one.</small>

In [293]:
model_2 = Sequential()
model_2.add(LSTM(32))
model_2.add(Dense(3, activation='softmax'))
input_shape = (None, X_train.shape[1], X_train.shape[2])
model_2.build(input_shape)
print(model_2.summary())

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_10 (LSTM)              (None, 32)                42624     
                                                                 
 dense_9 (Dense)             (None, 3)                 99        
                                                                 
Total params: 42,723
Trainable params: 42,723
Non-trainable params: 0
_________________________________________________________________
None


<font color="red">[TODO]</font> Compile your model

In [294]:
model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### 5. Train model

<font color="red">[TODO]</font> Train it!

In [295]:
history = model_2.fit(
    X_train, Y_train, 
    validation_data=(X_val, Y_val),
    epochs = 15,        # [TODO] how many iterations you want to run
    # initial_epoch = ?  # set this if you're continuing previous training
)

Epoch 1/15
15/15 [==============================] - 3s 66ms/step - loss: 1.0695 - accuracy: 0.4364 - val_loss: 0.9791 - val_accuracy: 0.6579
Epoch 2/15
15/15 [==============================] - 1s 36ms/step - loss: 0.9233 - accuracy: 0.6732 - val_loss: 0.7929 - val_accuracy: 0.7719
Epoch 3/15
15/15 [==============================] - 1s 35ms/step - loss: 0.6401 - accuracy: 0.8399 - val_loss: 0.4847 - val_accuracy: 0.8772
Epoch 4/15
15/15 [==============================] - 1s 36ms/step - loss: 0.4052 - accuracy: 0.8684 - val_loss: 0.4960 - val_accuracy: 0.8421
Epoch 5/15
15/15 [==============================] - 1s 37ms/step - loss: 0.3291 - accuracy: 0.8794 - val_loss: 0.3285 - val_accuracy: 0.8947
Epoch 6/15
15/15 [==============================] - 1s 35ms/step - loss: 0.2641 - accuracy: 0.9232 - val_loss: 0.3586 - val_accuracy: 0.8772
Epoch 7/15
15/15 [==============================] - 1s 36ms/step - loss: 0.2127 - accuracy: 0.9474 - val_loss: 0.3431 - val_accuracy: 0.9035
Epoch 8/15
15

### 6. Examine the result

In [296]:
testcases = [
    # 1
    'Green Beer Day (GBD) is a day-long party, where celebrants drink beer dyed green with artificial coloring or natural processes.',
    'When the siblings grew up, they held parties and introduced the tradition to friends while in college, and the tradition began to spread.',
    # 2
    'Politicians from the two main parties tend to win elections when not confronted by strong challengers from their own party (in which cases their traditional opponents tend to win).',
    'After the general election on 22 March 1992, five parties (Rassadorn, Justice Unity, Social Action, Thai Citizen, Chart Thai) designated Suchinda as the prime minister.',
    # 3
    'Typically, a party has the right to object in court to a line of questioning or at the introduction of a particular piece of evidence.',
    'In the practice of law, judicial estoppel (also known as estoppel by inconsistent positions) is an estoppel that precludes a party from taking a position in a case that is contrary to a position it has taken in earlier legal proceedings.'
]

In [297]:
# you must specify the padding width! 
test_X = process_text(testcases)

In [298]:
predictions = model_2.predict(test_X)

In [299]:
for idx, result in enumerate(predictions):
    predict_id = result.argmax()
    sense_id = predict_id + 1    # sense_id starts from 1
    print(testcases[idx])
    print(f'-> Sense {sense_id} (prob={result[predict_id]:.2f}): {SENSE[sense_id]}')
    print()

Green Beer Day (GBD) is a day-long party, where celebrants drink beer dyed green with artificial coloring or natural processes.
-> Sense 1 (prob=0.99): a social event at which a group of people meet to talk, eat, drink, dance, etc.

When the siblings grew up, they held parties and introduced the tradition to friends while in college, and the tradition began to spread.
-> Sense 1 (prob=0.96): a social event at which a group of people meet to talk, eat, drink, dance, etc.

Politicians from the two main parties tend to win elections when not confronted by strong challengers from their own party (in which cases their traditional opponents tend to win).
-> Sense 2 (prob=0.99): an organization of people with particular political beliefs

After the general election on 22 March 1992, five parties (Rassadorn, Justice Unity, Social Action, Thai Citizen, Chart Thai) designated Suchinda as the prime minister.
-> Sense 2 (prob=0.99): an organization of people with particular political beliefs

Typi

Yet again, the label might not be 100% correct, but it still should look reasonable.

## IV. Evaluation

We have our model built! It's time to see how good it is on the testing dataset.  
Get the predictions from the final model and examine the results.  

In [300]:
with open(os.path.join('.', 'party.test.txt'), 'r') as f:
    data = f.read().strip().split('\n')

# this dict maps sentence_id to the sentence itself
test_data = { sent_id: text for sent_id, text in [line.split('\t', 1) 
                                                 for line in data] }

In [301]:
len(test_data)

70

In [302]:
for idx, (sent_id, sentence) in enumerate(test_data.items()):
    if idx > 3: break
        
    print(f'{sent_id}: {sentence}')

1638: Patent ambiguity is that ambiguity which is apparent on the face of an instrument to any one perusing it, even if unacquainted with the circumstances of the parties.
1639: Smith played at parties, juke joints, and fish fries.
1640: Turkey has a multi-party system, with two or three strong parties and often a fourth party that is electorally successful.
1641: The Christian Liberation Movement ( or simply MCL) is a Cuban dissident party advocating political change in Cuba.


<font color="red">[TODO]</font> Get the labels of testing data.  

Try to reserve the sentence id, because you will need it while requesting your accuracy.  
Recommended format of `final_predictions` : 
```
{ sent_id: sense_id }
```

In [303]:
test=[]
for key,sentence in test_data.items():
    test.append(sentence)
test = process_text(test)

In [304]:
predictions = model_2.predict(test)

In [305]:
sense_id=[]
for idx, result in enumerate(predictions):
    predict_id = result.argmax() # select the index of the maximum value
    sense_id.append(predict_id + 1)    # sense_id starts from 1
    #print(testcases[idx])
    #print(f'-> Sense {sense_id} (prob={result[predict_id]:.2f}): {SENSE[sense_id]}')
    #print(sense_id)

In [306]:
final_predictions = {}
for idx, (sent_id, sentence) in enumerate(test_data.items()):
    final_predictions[sent_id]=int(sense_id[idx])

In [307]:
len(final_predictions)

70

In [308]:
for idx, (sent_id, pred) in enumerate(final_predictions.items()):
    if idx > 5: break
        
    print(sent_id, test_data[sent_id])
    print(f'-> Sense {pred}: {SENSE[pred]}')
    print()

1638 Patent ambiguity is that ambiguity which is apparent on the face of an instrument to any one perusing it, even if unacquainted with the circumstances of the parties.
-> Sense 3: a single entity which can be identified as one for the purposes of the law

1639 Smith played at parties, juke joints, and fish fries.
-> Sense 1: a social event at which a group of people meet to talk, eat, drink, dance, etc.

1640 Turkey has a multi-party system, with two or three strong parties and often a fourth party that is electorally successful.
-> Sense 2: an organization of people with particular political beliefs

1641 The Christian Liberation Movement ( or simply MCL) is a Cuban dissident party advocating political change in Cuba.
-> Sense 1: a social event at which a group of people meet to talk, eat, drink, dance, etc.

1642 Greens Party () was a green liberal party in Turkey.
-> Sense 2: an organization of people with particular political beliefs

1643 Under the Constitution of North Korea, 

In [309]:
final_predictions

{'1638': 3,
 '1639': 1,
 '1640': 2,
 '1641': 1,
 '1642': 2,
 '1643': 2,
 '1644': 1,
 '1645': 1,
 '1646': 1,
 '1647': 2,
 '1648': 2,
 '1649': 3,
 '1650': 1,
 '1651': 1,
 '1652': 1,
 '1653': 3,
 '1654': 2,
 '1655': 2,
 '1656': 3,
 '1657': 2,
 '1658': 2,
 '1659': 3,
 '1660': 2,
 '1661': 1,
 '1662': 2,
 '1663': 2,
 '1664': 3,
 '1665': 2,
 '1666': 2,
 '1667': 2,
 '1668': 3,
 '1669': 2,
 '1670': 1,
 '1671': 2,
 '1672': 2,
 '1673': 3,
 '1674': 3,
 '1675': 3,
 '1676': 3,
 '1677': 2,
 '1678': 3,
 '1679': 1,
 '1680': 1,
 '1681': 1,
 '1682': 3,
 '1683': 3,
 '1684': 3,
 '1685': 2,
 '1686': 1,
 '1687': 1,
 '1688': 1,
 '1689': 3,
 '1690': 2,
 '1691': 2,
 '1692': 1,
 '1693': 2,
 '1694': 2,
 '1695': 3,
 '1696': 3,
 '1697': 1,
 '1698': 2,
 '1699': 3,
 '1700': 2,
 '1701': 1,
 '1702': 1,
 '1703': 1,
 '1704': 3,
 '1705': 2,
 '1706': 3,
 '1707': 2}

### Get your accuracy

Send your predictions in json format to our server, and we will calculate the accuracy for you.  
The format should be 
```
{ sentence_id: sense_id }
```
Example,
```
{
    1001: 1,
    1002: 1,
    ...
}
```

In [225]:
import json
import requests

In [310]:
data = json.dumps(final_predictions)
ret = requests.post('http://jedi.nlplab.cc:4500/check', 
                    json = { 'data': data }
                   )

In [311]:
if not ret.ok:
    print('Something wrong :o')
print(ret.json())

{'accuracy': 0.7142857142857143, 'comment': ['Well done!']}


**REQUIREMENT**  
**Your accuracy should be <u>higher than 0.70</u> to get the full points.**

But do note that your assignment is mostly scored on your implementation, not just on the accuracy.  
So even if you brute-forcely attack our server and get 100% accuracy, you still can't get your points if your code doesn't make sense to TA.

## TA's note

Congratuation! You've finished the assignment this week.  
Don't forget to <b>[make an appoiment with TA](https://docs.google.com/spreadsheets/d/1QGeYl5dsD9sFO9SYg4DIKk-xr-yGjRDOOLKZqCLDv2E/edit#gid=1902646609) to demo/explain your implementation <u>before <font color="red">11/18 15:30</font></u></b> .  
Also make sure you submit your {student_id}.ipynb to [eeclass](https://eeclass.nthu.edu.tw/course/homework/4615).

Please note that <font color="red">we will announce our final project on 11/18</font>. Again, **we strongly suggest you join and listen** .  
We will have 2 Ph.D. students introduce the selected topics in class and give you some guidelines about how to approach your project.  
Also, we will have a team-matching session at the end of the class, in which you may want to participate to find teammates.